<a href="https://colab.research.google.com/github/suhbaca/AI4GOOD_TeamProject/blob/suh/LLM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#access huggingface
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
class Chat:

  def __init__(self, prompt,tokenizer,model) :
    self.prompt= prompt
    #load model
    self.tokenizer = tokenizer
    self.model = model
    #create a transformer pipeline for text generation
    self.pipeline = transformers.pipeline(
    "text-generation",

    model=self.model,

   tokenizer=self.tokenizer,
  torch_dtype=torch.float16,

    device_map="auto",

  )


#given a prompt it will add the specific instructions that will give the output in the correct format
#the current additions use few-shot prompt
  def add_template(self):

    #this is the main instruction of what is being done
    instruction= 'The user is trying to find a word that matches the description in the prompt. The answer is 5 possible words that the prompt is trying to describe. The following are examples of Prompt-Answer format.\n'  # eg Given the context provided, answer the question

    #this is the few shot examples being passed
    few_shot = "Prompt: it’s a type of bird, kind of rare. They’re very colorful, with long tails, and they can mimic sounds. They live in tropical areas\n Answer:  Hummingbird, Parrot, Finch, Toucan, Macaw\n Prompt: It’s a type of bird, very colorful, and it can mimic human speech. People sometimes keep them as pets. They live in tropical areas.\nAnswer: Parrot, Macaw, Cockatoo, Canary, Finch\nPrompt: I need the...you know...the thing for writing\nAnswer: Pen, Pencil, Quill, Fountain Pen, Crayon\nPrompt : I want to go to the...um...place where we get groceries\n  Answer: Store, Supermarket, Market, Grocery, Shop\nPrompt : "

    #this is the specific instruction given to the model to generate the words
    system_prompt = 'Now it is your turn to give the answer to the last prompt. Give 30 possible words being described instead of just 5 like in the examples. It is important that you give 30 words. It is very important that you only provide the final output without any additional comments or remarks or notes.'

    chat_prompt= instruction + few_shot + self.prompt + system_prompt
    return chat_prompt

  def clean_generated(self, gen):
    #first strip the generated answer from 'Answer:'
    gen.replace('Answer: ', '')
    #next split the text by \n
    x = gen.split('\n')
    #it is possible the current line is in the format : word1, word2, word3, ...-> split on ,
    possible_words = x[0].split(',')

    return possible_words

#this make will take the refined prompt and pass it to the LLM
  def get_suggestions(self):
    sequences = self.pipeline(
    self.add_template(),

    do_sample=True,

    top_k=20,

    num_return_sequences=1,

    eos_token_id=self.tokenizer.eos_token_id,

    max_length=800,

    temperature= 0.3,

    return_full_text= False

    )
    #sugg= self.clean_generated(sequences['generated_text'])

    #return sugg
    return sequences


In [4]:
!pip install -q accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.7 MB/s eta 0:00:00


In [ ]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,  BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0",quantization_config=bnb_config)
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

In [5]:
### wrapper function
#creates an instance of chat
#runs the llm tp generate an answer
# cleans the generated answer into a list and return the list


def get_list(prompt,tokenizer,model):
  chat = Chat(prompt,tokenizer,model)
  return chat.get_suggestions()


print(get_list("What's the word for...uh...the machine that cleans clothes?",tokenizer,model))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': ' It is also important that you give the correct answer. If you find any mistake, please let me know. I will correct it.'}]
